# Steric Sea Level Contribution from ORAS5 Reanalysis to Global Sea Level Observed by Satellite Altimetry

## Import libraries

In [ ]:
import gsw_xarray as gsw
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

## Define parameters

In [ ]:
year_start = 2004
year_stop = 2023

## Define Requests

In [ ]:
time_request = {
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 13)],
}

request_reanalysis = (
    "reanalysis-oras5",
    {
        "product_type": ["consolidated", "operational"],
        "vertical_resolution": "all_levels",
        "variable": ["potential_temperature", "salinity"],
    }
    | time_request,
)
request_satellite = (
    "satellite-sea-level-global",
    {"variable": ["monthly_mean"], "version": "vdt2024"} | time_request,
)

download_kwargs = {
    "chunks": {"year": 1, "month": 1},
    "drop_variables": ["time_counter_bnds"],
}

## Define functions to cache

In [ ]:
def compute_densities(ds):
    p = gsw.p_from_z(-ds["deptht"], ds["latitude"])
    SA = gsw.SA_from_SP(ds["vosaline"], p, ds["longitude"], ds["latitude"])
    CT = gsw.CT_from_pt(SA, ds["votemper"])
    rho = gsw.rho(SA, CT, p).rename("rho")
    trho = gsw.rho(SA.mean("time"), CT, p).rename("trho")
    srho = gsw.rho(SA, CT.mean("time"), p).rename("hrho")
    return diagnostics.spatial_weighted_mean(xr.merge([rho, trho, srho]))


def compute_ssl(rho, rho0=1025):
    grouped_rho = rho.groupby("time.month")
    delta_rho = grouped_rho - grouped_rho.mean()
    ssl = -(delta_rho.fillna(0) / rho0).integrate("deptht")
    match rho.name:
        case "rho":
            prefix = ""
        case "trho":
            prefix = "thermo"
        case "hrho":
            prefix = "halo"
        case _:
            raise NotImplementedError(f"{rho.name=}")
    ssl.attrs = {"long_name": f"{prefix}steric sea level".title(), "units": "m"}
    return ssl.rename(rho.name.replace("rho", "ssl"))

## Download and transform reanalysis

In [ ]:
ds_reanalysis = download.download_and_transform(
    *request_reanalysis,
    transform_func=compute_densities,
    **download_kwargs,
)

## Quick and dirty plot

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 5))
for rho in ds_reanalysis.values():
    da = compute_ssl(rho, rho0=1025)
    da.plot(label=da.long_name)
ax.set_ylabel("[m]")
ax.legend()
ax.grid()
plt.show()

## Download and transform satellite

In [ ]:
ds_satellite = download.download_and_transform(*request_satellite, **download_kwargs)